In [1]:
import os
import json
from tqdm import trange
import numpy as np
from tqdm import tqdm

In [2]:
path_dir = '/project/annotation/2D'
file_list1 = os.listdir(path_dir)

In [3]:
false_data = list()
true_data = list()

true_name = list()

for file_name1 in tqdm(file_list1):
    if(file_name1[0] == '.'):
        file_name1 = file_name1[2:]
    #print(file_name1)
    file_path = path_dir + '/' + file_name1
    
    if 'DS_Store' in file_path:
        continue

    file_list2 = os.listdir(file_path)

    if file_name1[0] == '6': # 쓰러짐 영상
        for file_name2 in file_list2:
            if 'DS_Store' in file_name2:
                continue
            if(file_name2[0] == '.'):
                file_name2 = file_name2[2:]
            read_file_path = file_path + '/' + file_name2
            true_name.append(file_name2)

            with open(read_file_path, 'r') as f:
                json_data = json.load(f)
                #print(json_data.keys())
                tmpList = list()

                for data in json_data['annotations']:
                    tmpList.append(data['keypoints'])

                true_data.append(tmpList)

    else: # 기타 영상
        for file_name2 in file_list2:
            if 'DS_Store' in file_name2:
                continue
            if(file_name2[0] == '.'):
                file_name2 = file_name2[2:]
            read_file_path = file_path + '/' + file_name2

    
            with open(read_file_path, 'r') as f:
                json_data = json.load(f)
                #print(json_data.keys())
                tmpList = list()
                for data in json_data['annotations']:
                    tmpList.append(data['keypoints'])

                false_data.append(tmpList)

100%|██████████| 697/697 [01:53<00:00,  6.17it/s]


In [4]:
for index, data in enumerate(true_data):
    if len(data) < 30:
        tmp_list = [[0 for w in range(48)] for h in range(30-len(data))]
        data += tmp_list
        true_data[index]=data
    else:
        true_data[index] = data[:30]

In [5]:
for index, data in enumerate(false_data):
    if len(data) < 30:
        tmp_list = [[0 for w in range(48)] for h in range(30-len(data))]
        data += tmp_list
        false_data[index]=data
    else:
        false_data[index] = data[:30]

In [ ]:
true_np = np.array(true_data, dtype=np.float32)
false_np = np.array(false_data, dtype=np.float32)

In [11]:
del(true_data)
del(false_data)

print(false_np.shape)
print(true_np.shape)

(793340, 30, 48)
(15324, 30, 48)


In [12]:
y_true = np.array([1 for w in range(len(true_np))])
y_false = np.array([0 for w in range(len(false_np))])

In [13]:
data_x = np.concatenate((false_np, true_np), axis=0)
data_y = np.concatenate((y_false, y_true), axis=0)

In [14]:
np.count_nonzero(np.isnan(data_x))

64

In [ ]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(data_x)*0.8)
train_x, remaining_x = data_x[:split_idx], data_x[split_idx:]
train_y, remaining_y = data_y[:split_idx], data_y[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 16

# make sure to SHUFFLE for your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [ ]:
train_data = torch.load('train.pt')
test_data = torch.load('test.pt')
valid_data = torch.load('valid.pt')